In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc='My bar!')
from collections import defaultdict

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [28]:
genes = pd.read_csv("Homo_sapiens.GRCh38.108.gtf", sep = "\t", comment="#", header=None)
genes[0] = "chr" + genes[0].astype(str)


def find_genes(row):
    gene_df = genes[(genes[0] == row['CHROM']) & (genes[3] <= row['POS']) & 
                    (genes[4] >= row['POS'])]
    if len(gene_df) == 0:
        return None
    gene_df_info = gene_df[8].str.split(";")
    gene_set = set()
    for info in list(gene_df_info):
        for field in info:
            if "gene_name" in field:
                name = field.strip().replace("gene_name ","").replace('"','')
                gene_set.add(name)
    if len(gene_set) == 0:
        return None
                
    return ",".join(list(gene_set))

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
all_expansions = pd.DataFrame(columns = ['CHROM', 'POS', 'PERIOD', 'MOTIF', 'outlier_threshold', 'AFR_freq',
       'NON_AFR_freq'])
for i in range(1,23):
    expansions = pd.read_csv(f"{i}_expansions.csv")
    all_expansions = pd.concat([expansions, all_expansions])
    
    
all_expansions['gene'] = all_expansions.apply(lambda row: find_genes(row), axis = 1)

In [36]:
significant_expansion = all_expansions[(all_expansions['outlier_threshold'] > 10) & 
               (all_expansions['PERIOD'] > 2)].sort_values('outlier_threshold', ascending = False)

significant_expansion.to_csv("significant_expansions.csv",  index = False)